In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import functools




In [ ]:
# Data file paths
alcohol_path = "data (1).csv"
happiness_path = "2018.csv"
hdi_path = "2018 Human Development Index (HDI).csv"
gdp_path = "GDP per capita (2017 PPP $).csv"
life_path = "Life expectancy at birth (years).csv"

# Output file
output_data_file = "output_data/alcohol_vs_happiness.csv"

# Read the data files
alcohol_data = pd.read_csv(alcohol_path)
happiness_data = pd.read_csv(happiness_path)
hdi_data = pd.read_csv(hdi_path, skiprows=5, encoding="cp1252")
gdp_data = pd.read_csv(gdp_path, encoding="cp1252", skiprows=5)
life_data = pd.read_csv(life_path, encoding="cp1252", skiprows=6)


In [ ]:
# Grab 2018 data, clean and merge HDI, GDP and Life Expectancy 
clean_gdp_data = gdp_data[["Country", "2018"]]
gdp_hdi = clean_gdp_data.merge(hdi_data, how='inner', on=["Country"])
final_gdp_hdi = gdp_hdi.rename(columns={"2018_x": "GDP (per capita PPP)", "2018_y": "HDI"})
clean_life_data = life_data[["Country", "2018"]]
clean_life_data = clean_life_data.rename(columns={"2018": "Life Expectancy"})
final_hdi_gdp_life = final_gdp_hdi.merge(clean_life_data, how='inner', on=["Country"])
#final_hdi_gdp_life = final_hdi_gdp_life.dropna()
final_hdi_gdp_life = final_hdi_gdp_life.iloc[:-25]
final_hdi_gdp_life.dtypes

In [ ]:
final_hdi_gdp_life[["GDP (per capita PPP)", "HDI Rank", "HDI", "Life Expectancy"]] = final_hdi_gdp_life[["GDP (per capita PPP)", "HDI Rank", "HDI", "Life Expectancy"]].astype(float)
final_hdi_gdp_life.dtypes


In [ ]:
final_hdi_gdp_life = final_hdi_gdp_life.sort_values(by='GDP (per capita PPP)', ascending=False)

final_hdi_gdp_life["Country"] = final_hdi_gdp_life["Country"].str.strip()
final_hdi_gdp_life

In [ ]:
top_hdi = final_hdi_gdp_life.sort_values(by='Country', ascending=False)
top_hdi.head(50)

In [ ]:
# Clean World Happiness Report
happiness_final = happiness_data.rename(columns={"Country or region": "Country"})
happiness_final['Country'] = happiness_final['Country'].replace({"Russia": "Russian Federation"})
happiness_final

In [ ]:
top_happ = happiness_final.sort_values(by='Country', ascending=False)
top_happ.head(50)

In [ ]:
# Clean, pivot and reorganize alcohol data
alcohol_df = alcohol_data.drop([0])
alcohol_results = alcohol_df.pivot_table('Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)', 
                                           ["Unnamed: 0", "Unnamed: 1"], 'Unnamed: 2')                              
alcohol = pd.DataFrame(alcohol_results.to_records())
alcohol_set = alcohol[["Unnamed: 0", "All types", "Beer", "Spirits", "Wine", "Other alcoholic beverages"]]
alcohol_final = alcohol_set.rename(columns={"Unnamed: 0": "Country", "Other alcoholic beverages": "Other"})
alcohol_final

In [ ]:
#alcohol_final["Country"]=="United States of America"
alcohol_final['Country'] = alcohol_final['Country'].replace({'United States of America': 'United States',
                                                            "United Kingdom of Great Britain and Northern Ireland":
                                                            "United Kingdom", "Russia": "Russian Federation"})

In [ ]:
top_alcohol = alcohol_final.sort_values(by='Country', ascending=False)
top_alcohol.head(50)

In [ ]:
# Combine World Happiness Report and WHO alcohol data
happiness_alcohol = happiness_final.merge(alcohol_final, how='inner', on=["Country"])

# Display the data table for preview
happiness_alcohol["Country"] = happiness_alcohol["Country"].str.strip()

happiness_alcohol


In [ ]:
top_happy = happiness_alcohol.sort_values(by='Country', ascending=False)
top_happy.head(50)

In [ ]:
final_results = final_hdi_gdp_life.merge(happiness_alcohol, how="inner", on=["Country"])
final_results.to_csv(output_data_file)

In [ ]:
final_results

In [ ]:
top_alcohol = final_results.sort_values(by='All types', ascending=False)
top_alcohol.head(50)

In [ ]:
all_types_df = final_results[['Country',
                              'All types',
                              'Beer',
                              'Spirits',
                              'Wine',
                              'Other']]

# get min and max of all types
all_types_max = all_types_df['All types'].max()
all_types_min = all_types_df['All types'].min()
# print(f'max: {all_types_max}')
# print(f'min: {all_types_min}')

# find range of all types
all_types_range = all_types_max - all_types_min
# print(f'range: {all_types_range}')

# split range into 3 even groups using an increment
all_types_bin_increments = all_types_range / 3
# print(f'increments: {all_types_bin_increments}')

# create all types bins
all_types_bins = [all_types_min,
               all_types_min + all_types_bin_increments,
               all_types_min + (all_types_bin_increments * 2),
               all_types_max]
# print(all_types_bins)

# turn bin increments into strings for labels
bin_str_min = str(all_types_min)
bin_str_1 = str(all_types_min + all_types_bin_increments)
bin_str_2 = str(all_types_min + (all_types_bin_increments * 2))
bin_str_max = str(all_types_max)

# create bin labels
all_tyles_bin_labels = [f'Small: ({bin_str_min}-{bin_str_1})',
                        f'Medium: ({bin_str_1}-{bin_str_2})',
                        f'Large: ({bin_str_2}-{bin_str_max})']
print(all_tyles_bin_labels)

# name bin and separate rows into bins
all_types_df['Groups by Total Consumption'] = pd.cut(all_types_df['All types'],all_types_bins, labels=all_tyles_bin_labels, include_lowest=True)
all_types_df

# create group by bin
all_types_grouped_df = all_types_df.groupby('Groups by Total Consumption')
all_types_grouped_df.count()

# calculate averages by category
group_country_count = all_types_grouped_df['Country'].count()
group_avg_all = all_types_grouped_df['All types'].mean()
group_avg_beer = all_types_grouped_df['Beer'].mean()
group_avg_spirits = all_types_grouped_df['Spirits'].mean()
group_avg_wine = all_types_grouped_df['Wine'].mean()
group_avg_other = all_types_grouped_df['Other'].mean()

# create new grouped df
group_avg_df = pd.DataFrame({'Count of Countries': group_country_count,
                             'Average All Types': round(group_avg_all, 2),
                             'Average Beer': round(group_avg_beer, 2),
                             'Average Spirits': round(group_avg_spirits, 2),
                             'Average Wine': round(group_avg_wine, 2),
                             'Average Other': round(group_avg_other, 2)})

group_avg_df.reset_index(inplace=True)

group_avg_df

In [ ]:
# Setting the positions and width for the bars
pos = list(range(len(group_avg_df['Count of Countries'])))
width = 0.2

# plot bars
fig, ax = plt.subplots(figsize=(12,7))
plt.bar(pos, 
        group_avg_df['Average Beer'], 
        width,
        alpha=0.75, 
        color='#FFC222',
        label=group_avg_df['Groups by Total Consumption'][0]) 

plt.bar([p + width for p in pos], 
        group_avg_df['Average Spirits'],
        width, 
        alpha=0.75, 
        color='#F78F1E', 
        label=group_avg_df['Groups by Total Consumption'][1]) 

plt.bar([p + width*2 for p in pos], 
        group_avg_df['Average Wine'],
        width, 
        alpha=0.75, 
        color='#800020', 
        label=group_avg_df['Groups by Total Consumption'][2])

plt.bar([p + width*3 for p in pos], 
        group_avg_df['Average Other'],
        width, 
        alpha=0.75, 
        color='#696969', 
        label=group_avg_df['Groups by Total Consumption'][2]) 

# format plot
plt.ylabel('Litres of Pure Alcohol per Year', fontsize=14)
plt.title('Countries Grouped by Total Consumption', fontsize=18)
plt.xticks([p + 1.5 * width for p in pos])
ax.set_xticklabels(group_avg_df['Groups by Total Consumption'], fontsize=12)
plt.xlim(min(pos)-width, max(pos)+width*4)
plt.tick_params(axis='y', which='major', labelsize=12)
plt.ylim([0, 5])
plt.legend(['Beer', 'Spirits', 'Wine', 'Other'], loc='upper left', fontsize=12)
plt.grid(linestyle=':', axis='y')

plt.savefig('output_data/groupedbar.png')
plt.show()

In [ ]:
all_alc_x = final_results['All types']
hdi_y = final_results['HDI']

hdi_slope, hdi_int, hdi_r, hdi_p, hdi_std_err = st.linregress(all_alc_x, hdi_y)
hdi_regression = hdi_slope * all_alc_x + hdi_int
hdi_regression_txt = f'y = {str(round(hdi_slope,2))}x + {str(round(hdi_int,2))}'

plt.figure(figsize=(12, 7))
plt.scatter(all_alc_x, hdi_y, marker='o', facecolors="#648DE5", edgecolors="#1e4bae", s=100)
plt.plot(all_alc_x, hdi_regression, color='#B8AB3E', linewidth=3)
plt.title(f'Total Consumption vs. Human Development Index (HDI)', fontsize=18)
plt.xlabel('Total Consumption (litres)', fontsize=14)
plt.ylabel('Human Development Index', fontsize=14)
plt.annotate(f'Correlation:  {round(hdi_r, 2)}', xy=(0.60, 0.1), xycoords='axes fraction', \
             fontsize=14, color='#997625')
plt.annotate(f'Linear Reg:  {hdi_regression_txt}', xy=(0.60, 0.05), xycoords='axes fraction', \
             fontsize=14, color='#997625')
plt.tick_params(axis='both', which='major', labelsize=12)

plt.savefig('output_data/hdi_v_all_scatter.png')
plt.show()


In [ ]:
y_values = final_results["GDP (per capita PPP)"] 
x_values = final_results["All types"]

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create scatterplot with linear regression
plt.scatter(x_values,y_values)

plt.plot(x_values,regress_values, "r-")
plt.annotate(line_eq,(5, 10000),fontsize=15,color="red")
plt.xlabel("Alcohol Consumption")
plt.ylabel("GDP (per capita PPP)")
print(f"The Correlation Coefficient is: {rvalue}")
plt.title("GDP vs Alcohol Consumption")
plt.grid()
plt.show()


In [ ]:
x_values = final_results["GDP (per capita PPP)"]
y_values = final_results["Score"]

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create scatterplot with linear regression
plt.scatter(x_values,y_values)

plt.plot(x_values,regress_values, "r-")
plt.annotate(line_eq,(60000,4),fontsize=15,color="red")
plt.xlabel("GDP (per capita PPP)")
plt.ylabel("Happiness Score")
print(f"The Correlation Coefficient is: {rvalue}")
plt.title(" Happiness Score vs GDP")
plt.grid()
plt.show()

In [ ]:
x_values = final_results["Score"]
y_values = final_results["All types"]

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create scatterplot with linear regression
plt.scatter(x_values,y_values)

plt.plot(x_values,regress_values, "r-")
plt.annotate(line_eq,(6,4),fontsize=15,color="red")
plt.xlabel("Happiness Score")
plt.ylabel("Alcohol Consumption")
print(f"The Correlation Coefficient is: {rvalue}")
plt.title(" Happiness Score vs GDP")
plt.grid()
plt.show()

In [ ]:
x_values = final_results["All types"]
y_values = final_results["Score"]

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create scatterplot with linear regression
plt.scatter(x_values,y_values)

plt.plot(x_values,regress_values, "r-")
plt.annotate(line_eq,(3,4),fontsize=15,color="red")
plt.xlabel("Alcohol Consumption")
plt.ylabel("Happiness Score")
print(f"The Correlation Coefficient is: {rvalue}")
plt.title(" Happiness Score vs GDP")
plt.grid()
plt.show()

In [ ]:
x_values = final_results["Life Expectancy"]
y_values = final_results["All types"]

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create scatterplot with linear regression
plt.scatter(x_values,y_values)

plt.plot(x_values,regress_values, "r-")
plt.annotate(line_eq,(70,4),fontsize=15,color="red")
plt.xlabel("Life Expectancy")
plt.ylabel("Alcohol Consumption")
print(f"The Correlation Coefficient is: {rvalue}")
plt.title("Live Expectancy vs GDP")
plt.grid()
plt.show()

In [ ]:
x_values = final_results["HDI"]
y_values = final_results["All types"]

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create scatterplot with linear regression
plt.scatter(x_values,y_values)

plt.plot(x_values,regress_values, "r-")
plt.annotate(line_eq,(1,.5),fontsize=15,color="red")
plt.xlabel("HDI")
plt.ylabel("Alcohol Consumption")
print(f"The Correlation Coefficient is: {rvalue}")
plt.title("Alcohol Consumption vs HDI")
plt.grid()
plt.show()